In [1]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
import matplotlib.pyplot as plt
import hvplot as hvp
import datetime
import altair as alt
import seaborn as sns

### DataFrame for Drawdown

In [2]:
# Creating a range of dates
start_date = datetime.datetime(2018, 1, 1)
end_date = datetime.datetime(2023, 12, 31)
date_range = pd.date_range(start_date, end_date, freq='D')
# Dates df
dates_df = pd.DataFrame(date_range, columns=['Date'])
#print(dates_df.head(1825))

# Creating a range of random "prices"
portfolio_df = pd.DataFrame(
    {
    "fake1": np.random.uniform(1000, -500, 100),
    "fake2": np.random.uniform(1000, -500, 100),
    "fake3": np.random.uniform(1000, -500, 100)
    }
    )
f1_portfolio_df = portfolio_df["fake1"].to_frame()

# Concatinating and finalizing test data
returns = pd.concat([dates_df, portfolio_df], join = "inner", axis = 1)
returns.set_index('Date', inplace = True)
returns = pd.DataFrame(returns)
returns = returns.reset_index().rename(columns={'Date': 'date'})
returns.set_index('date', inplace = True)

### Data Frame for Portfolio Pie Chart

In [3]:
# Creating DataFrame for Bitcoin
btc_portfolio_df = pd.DataFrame({
    'stock': ['BTC:USD'],
    'position_size': np.random.uniform(10, 150, size = 1)
})

# Creating DataFrame for Stocks
stock_portfolio_df = pd.DataFrame({
    'stock': ['AMC', 'VSCO', 'TSLA', 'GOOGL'],
    'position_size': np.random.uniform(1000, 10000, 4)  # for 5 stocks
})

# Combining DataFrames
size_portfolio_df = pd.concat([btc_portfolio_df, stock_portfolio_df], axis = 0).reset_index(drop = True)

# Renaming stock column to securities
size_portfolio_df = size_portfolio_df.rename(columns = {'stock' : 'securities'})

### Portfolio Pie Chart Function

In [4]:
# Defining Portfolio pie chart function
def portfolio_piechart (holdings, size):
    selection = alt.selection_single()
    pie_chart = alt.Chart(size_portfolio_df).mark_arc(innerRadius=50).encode(
        theta = alt.Theta(field = 'position_size', type = 'quantitative'),
        color = alt.Color(field = 'securities', type = 'nominal'),
        tooltip=['securities', 'position_size']
    )
    return pie_chart

### Portfolio Pie Chart

In [5]:
one = size_portfolio_df['securities']
two = size_portfolio_df['position_size']
user_in_holdings = portfolio_piechart(one, two)
user_in_holdings

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)

In [6]:
initial_investment = 10000

In [7]:
def cumulative_returns(df):
    return (1 + df).cumprod()

In [8]:
user_cr = cumulative_returns(portfolio_df)
user_cr

,fake1,fake2,fake3
0,-135.89,705.10,640.62
1,-80726.33,19971.65,170370.99
2,-37443929.75,14759436.89,-13231.25
3,6027217523.35,9445706506.40,989537.11
4,1309930597665.32,736496840439.65,-235210997.44
...,...,...,...
95,-1373370543460402500555530064617655452378433984...,-2162356010649305493990049589565027705342404278...,-1506423872740697538596604252817618537453543155...
96,-5848523490661315328286849571893596896251911338...,64097529628278542208283764535611381430960450131...,18508692806027938781502175852445660823658550698...
97,-1667138695252075872870948924275306949947279363...,53214654516177641126291439690745987743444931104...,35118940152444541201109131219545796903811445981...
98,22539072695639665607791649646816445875122548795...,13665220203770980731375387424876289786054748010...,23285246419080999005526007692217860176186904581...


### Correlation

In [9]:
returns_cor = returns.corr()
returns_cor

,fake1,fake2,fake3
fake1,1.00,-0.06,0.19
fake2,-0.06,1.00,0.05
fake3,0.19,0.05,1.00


### Correlation Plot Function

In [63]:
#In the works
# def correlation_chart(returns):
#     returns_cor = returns.corr()
#     corr_chart= sns.heatmap(returns_cor, vmin=-1 , vmax = 1)
#     return corr_chart

def correlation_chart(df, quant_x,  quant_y):
    corr_chart = alt.Chart(df).mark_circle(size = 60).encode(
        x=alt.X(f'{quant_x}:Q', title = "Asset_1"),
        y=alt.Y(f'{quant_y}:Q', title = "Asset_2"),
        tooltip= [quant_x, quant_y]
    ).interactive()
    return returns_cor

### Correlation Chart

In [64]:
correlation_chart(returns, 'fake1', 'fake2')
correlation_chart

<function __main__.correlation_chart(df, quant_x, quant_y)>

### Covariance

In [12]:
def covariance(df, ticker, market):
    return df[ticker].cov(df[market])

In [13]:
user_covariance = covariance(portfolio_df, 'fake2', 'fake3')
user_covariance

9462.50408220147

### Variance

In [14]:
def variance(df, market):
    return df[market].var()

In [15]:
user_variance = variance(portfolio_df, 'fake3')
user_variance

186145.82437766952

### Drawdown

In [16]:
def drawdown(df, tickers):
    Roll_Max = df[tickers].cummax()
    Daily_Drawdown = df[tickers]/Roll_Max - 1.0
    Max_Daily_Drawdown = Daily_Drawdown.cummin() * 100

In [17]:
user_drawdown = drawdown(portfolio_df, 'fake1')
user_drawdown

### Drawdown Chart Function

In [18]:
def drawdown_chart(df, date, quant):
    '''
    df: dataframe
    date: str: should be a date column name
    quant: str: should be a quantitative data type
    '''
    chart = alt.Chart(df.reset_index()).mark_bar().encode(
        x = date + ':T',  # T specifies temporal data type for x-axis
        y = quant + ':Q',  # Q specifies a quantitative data type for y-axis
        color = alt.condition(
            getattr(alt.datum, quant) > 0,
            alt.value("Green"),  # The positive color
            alt.value("red")
    ), tooltip= [date + ':T', quant + ':Q']
    ).properties(width=800)
    chart = chart.interactive()
    return chart

In [19]:
user_drawdown = drawdown_chart(returns, 'date', 'fake1')
user_drawdown

alt.Chart(...)

In [20]:
def tracking_error(df, tickers, market):
    track_error = np.sqrt(sum([i**2 for i in df[tickers] - df[market]]))
    return track_error

In [21]:
user_tracking_error = tracking_error(portfolio_df, 'fake2', 'fake3')
user_tracking_error

5983.29857357601

### Beta

In [22]:
def beta(covariance, variance):
    chart_beta = covariance / variance
    chart_beta = chart_beta
    return chart_beta

In [23]:
user_beta = beta(user_covariance, user_variance)
user_beta

0.05083382404003372

### Beta chart function

In [24]:
# In the works
# def beta_chart(covariance, variance):
#     chart_beta = covariance / variance
#     chart_beta = chart_beta.rolling(window = 21).plot()
#     plt.show()
#     beta.shape
#     chart_beta.plot()
#     return chart_beta

### Sharpe Ratio

In [25]:
def sharpe_ratio(df):
    sharpe = (df.mean()*252) / (df.std() * np.sqrt(252))
    return sharpe

In [26]:
user_sharpe_ratio = sharpe_ratio(portfolio_df)
user_sharpe_ratio

fake1   8.05
fake2   9.13
fake3   7.79
dtype: float64

### Return on Investment

In [27]:
def return_on_investment(investment, returns):
    cumulative_profit = investment * returns
    return_oi = (cumulative_profit - investment) / investment
    return(return_oi)

### Return on Investment

In [28]:
def return_on_investment(investment, returns):
    cumulative_profit = investment * returns
    return_oi = (cumulative_profit - investment) / investment
    return(return_oi)

In [29]:
user_roi = return_on_investment(initial_investment, user_cr)
user_roi

,fake1,fake2,fake3
0,-136.89,704.10,639.62
1,-80727.33,19970.65,170369.99
2,-37443930.75,14759435.89,-13232.25
3,6027217522.35,9445706505.40,989536.11
4,1309930597664.32,736496840438.65,-235210998.44
...,...,...,...
95,-1373370543460402500555530064617655452378433984...,-2162356010649305493990049589565027705342404278...,-1506423872740697538596604252817618537453543155...
96,-5848523490661315328286849571893596896251911338...,64097529628278542208283764535611381430960450131...,18508692806027938781502175852445660823658550698...
97,-1667138695252075872870948924275306949947279363...,53214654516177635203904918157890247581627424457...,35118940152444541201109131219545796903811445981...
98,22539072695639665607791649646816445875122548795...,13665220203770980731375387424876289786054748010...,23285246419080999005526007692217860176186904581...


### ROI Chart

In [30]:
# In the works
# source = data.wheat()

# bar = alt.Chart(source).mark_bar().encode(
#     x = df + ':O',
#     y = returns + ':Q'
# )

# <!-- line = alt.Chart(source).mark_line(color='red').transform_window(
#     # The field to average
#     rolling_mean='mean(wheat)',
#     # The number of values before and after the current value to include.
#     frame=[-9, 0]
# ).encode(
#     x='year:O',
#     y='rolling_mean:Q'
# )

# (bar + line).properties(width=600) -->